# Complete Animal Detection Training Script for Google Colab

This notebook trains a custom YOLOv8 model for animal detection.

**Classes:** bison, cow, deer, donkey, elephant, fox, goat, hippopotamus, hyena, kangaroo, leopard, lion, ox, panda, rhinoceros, tiger, wolf

**Training:** 300 epochs with YOLOv8n model


In [ ]:
# Install dependencies
!pip install ultralytics opencv-python scikit-learn pyyaml
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
print("✅ Dependencies installed successfully!")

Defaulting to user installation because normal site-packages is not writeable
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nv

In [ ]:
import torch, os
from pathlib import Path
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("Using CPU")
!mkdir -p animals_dataset yolo_dataset
print("✅ Setup completed!")

In [ ]:
# Download sample dataset (optional)
import urllib.request, os
animal_classes = ['bison','cow','deer','donkey','elephant','fox','goat','hippopotamus','hyena','kangaroo','leopard','lion','ox','panda','rhinoceros','tiger','wolf']
for animal in animal_classes:
    os.makedirs(f'animals_dataset/{animal}', exist_ok=True)
sample_urls = {
    'bison':['https://images.unsplash.com/photo-1552084117-56a987666449?w=400'],
    'cow':['https://images.unsplash.com/photo-1546445317-29d45416c916?w=400'],
    'deer':['https://images.unsplash.com/photo-1552083375-1447ce886485?w=400'],
    'lion':['https://images.unsplash.com/photo-1549366021-9f761d450615?w=400'],
    'tiger':['https://images.unsplash.com/photo-1561731216-c3a4d99437d5?w=400'],
    'elephant':['https://images.unsplash.com/photo-1552083375-1447ce886485?w=400'],
    'panda':['https://images.unsplash.com/photo-1552083375-1447ce886485?w=400'],
    'wolf':['https://images.unsplash.com/photo-1552083375-1447ce886485?w=400']}
for animal,urls in sample_urls.items():
    for i,url in enumerate(urls):
        try:
            filename=f'animals_dataset/{animal}/{animal}_{i+1}.jpg'
            urllib.request.urlretrieve(url, filename)
            print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Failed {url}: {e}")
print("✅ Sample dataset downloaded!")

In [ ]:
# Upload your own dataset (alternative)
from google.colab import files
import zipfile, os
print("Upload your dataset zip file:")
uploaded = files.upload()
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename,'r') as zip_ref:
            zip_ref.extractall('animals_dataset')
        print(f"Extracted {filename} to animals_dataset/")
        break
!find animals_dataset -type d | head -20

In [ ]:
# Prepare dataset
import os, shutil, random, yaml, cv2, numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
class DatasetPreparator:
    def __init__(self,dataset_path="animals_dataset",output_path="yolo_dataset"):
        self.dataset_path=Path(dataset_path)
        self.output_path=Path(output_path)
        self.output_path.mkdir(exist_ok=True)
        self.animal_classes={'bison':0,'cow':1,'deer':2,'donkey':3,'elephant':4,'fox':5,'goat':6,'hippopotamus':7,'hyena':8,'kangaroo':9,'leopard':10,'lion':11,'ox':12,'panda':13,'rhinoceros':14,'tiger':15,'wolf':16}
        self.class_names={v:k for k,v in self.animal_classes.items()}
    def prepare_dataset(self):
        yolo_dataset=self.output_path
        for s in ["train","val","test"]:
            (yolo_dataset/"images"/s).mkdir(parents=True,exist_ok=True)
            (yolo_dataset/"labels"/s).mkdir(parents=True,exist_ok=True)
        image_data=[]
        for animal in self.animal_classes:
            animal_path=self.dataset_path/animal
            if animal_path.exists():
                cid=self.animal_classes[animal]
                for f in animal_path.glob("*.jpg"):
                    image_data.append({'image_path':f,'class_id':cid,'class_name':animal})
        if not image_data:
            print("No images found")
            return None
        train_data,temp=train_test_split(image_data,train_size=0.8,random_state=42,stratify=[d['class_id'] for d in image_data])
        val_data,test_data=train_test_split(temp,train_size=0.5,random_state=42,stratify=[d['class_id'] for d in temp])
        for split,data in [("train",train_data),("val",val_data),("test",test_data)]:
            for item in data:
                dest_image=yolo_dataset/"images"/split/item['image_path'].name
                shutil.copy2(item['image_path'],dest_image)
                label_file=yolo_dataset/"labels"/split/(item['image_path'].stem+".txt")
                with open(label_file,'w') as f:
                    f.write(f"{item['class_id']} 0.5 0.5 1.0 1.0\n")
        config={'path':str(yolo_dataset.absolute()),'train':'images/train','val':'images/val','test':'images/test','nc':len(self.animal_classes),'names':list(self.animal_classes.keys())}
        with open(yolo_dataset/"dataset.yaml",'w') as f: yaml.dump(config,f)
        return yolo_dataset
preparator=DatasetPreparator()
yolo_dataset=preparator.prepare_dataset()

In [ ]:
# Train model
from ultralytics import YOLO
import json
from pathlib import Path
def train_model(model_size="n",epochs=300,batch_size=16):
    config_file=Path("yolo_dataset/dataset.yaml")
    if not config_file.exists():
        print("Dataset not found")
        return None
    model=YOLO(f'yolov8{model_size}.pt')
    train_args={'data':str(config_file),'epochs':epochs,'batch':batch_size,'imgsz':640,'patience':50,'save':True,'save_period':25,'cache':True,'device':'auto','workers':4,'project':'animal_detection_training','name':f'yolov8{model_size}_animals_300epochs','exist_ok':True}
    results=model.train(**train_args)
    model_info={'model_size':model_size,'epochs':epochs,'batch_size':batch_size,'classes':preparator.animal_classes,'class_names':preparator.class_names,'training_results':str(results)}
    with open(f"model_info_{model_size}_300epochs.json",'w') as f: json.dump(model_info,f,indent=2)
    return model
trained_model=train_model()

In [ ]:
# Evaluate model
if 'trained_model' in locals() and trained_model:
    results=trained_model.val()
    print(f"mAP50: {results.box.map50:.3f}, mAP50-95: {results.box.map:.3f}")
else:
    print("No trained model")

In [ ]:
# Test on sample image
import cv2
from pathlib import Path
if 'trained_model' in locals() and trained_model:
    test_images=list(Path("yolo_dataset/images/test").glob("*.jpg"))
    if test_images:
        sample=str(test_images[0])
        results=trained_model(sample)
        for r in results:
            for box in r.boxes:
                cid=int(box.cls[0]);conf=float(box.conf[0])
                print(preparator.class_names.get(cid),conf)
        cv2.imwrite("test_result.jpg",results[0].plot())

In [ ]:
# Download model
import json
if 'trained_model' in locals() and trained_model:
    !zip -r animal_detection_model.zip animal_detection_training/
    with open("model_info_n_300epochs.json") as f: info=json.load(f)
    print(info)

In [ ]:
# Test uploaded image
from google.colab import files
if 'trained_model' in locals() and trained_model:
    uploaded=files.upload()
    image_path=list(uploaded.keys())[0]
    results=trained_model(image_path)
    cv2.imwrite("uploaded_test_result.jpg",results[0].plot())

In [ ]:
# Export model
if 'trained_model' in locals() and trained_model:
    trained_model.export(format='onnx')
    try:
        trained_model.export(format='engine')
    except Exception as e:
        print(e)
    try:
        trained_model.export(format='coreml')
    except Exception as e:
        print(e)

## Usage Instructions
1. Run cells in order.
2. Enable GPU in Colab (Runtime → Change runtime type → GPU).
3. Train will take 2–4 hours.
4. Troubleshooting:
   - Reduce batch size if OOM.
   - Ensure dataset present.
   - Check GPU enabled.
